## Planning

Planning helps me stay organized and make sure I complete tasks on time. It also lets me find problems early and make better decisions for a smooth project.

- **Sprint 4 Plan**: In this plan, I layout everything I wanted to do and my ideas. This helps me stay focused on my specific task.

<img src="https://i.imgur.com/moEzDq4.png" alt="Project Screenshot" width="600">

Link to issue: <a href="https://github.com/tarunrayavarapu/tarun_2025/issues/10" target="_blank">Sprint 4</a>


- **Figma**: Creating a figma was helpful for me because it layed out the design of what I wanted my feature to look like, and also gave me a functionality idea of what I wanted my feature to be exactly.

<img src="https://i.imgur.com/Oe8Vtal.png" alt="Project Screenshot" width="600">

Link to Figma: <a href="https://www.figma.com/design/RuSyBpijlRXUBccE5ye1SF/Travel-Social-Media?node-id=0-1&p=f&t=qh7mTYJsBl2d8xGg-0" target="_blank">Figma</a>

---


## Kanban Board/Burndown List

Creating a Kanban board is helpful because it visually organizes tasks, making it easier to track progress, identify bottlenecks, and improve workflow efficiency. It provides a clear overview of tasks in different stages (To-Do, In Progress, Done), helping me stay organized, prioritize work, and collaborate effectively.

- **Kanban Board**: As you can see in the Kanban Board, everything was already done, helping us visual what we have completed so far

<img src="https://i.imgur.com/3Xilhvh.png" alt="Kanban Board Screenshot" width="600">

link to kanban: <a href="https://github.com/users/Kiruthic-Selvakumar/projects/2/views/1" target="_blank">Kanban Board</a>


- **Burndown List**: As you can see in the screenshot, having a burndown list was super useful too because it visualized what tasks I still need to do feature wise and what improvements I needed to make. 

<img src="https://i.imgur.com/SM5E1xo.png" alt="Project Screenshot" width="600">

link to Burndown list: <a href="https://github.com/users/Kiruthic-Selvakumar/projects/2/views/1?pane=issue&itemId=89573585&issue=Kiruthic-Selvakumar%7Ctravel_frontend%7C4" target="_blank">Burndown list</a>

---


## Making an API

- **Post**: I implemented the POST method to allow only authenticated users to submit a food review. Using @token_required(), I ensure that every review is tied to a specific user, preventing unauthorized submissions. I also validate the input, making sure the request contains all necessary fields (food, review, and rating). If everything is correct, the review is stored in the database and returned as JSON.

In [ ]:
@token_required()
def post(self):
    current_user = g.current_user
    data = request.get_json()

    if not data or 'food' not in data or 'review' not in data or 'rating' not in data:
        return {'message': 'Invalid, data required'}, 400

    food_review = FoodReview12345(
        food=data.get('food'),
        review=data.get('review'),
        rating=data.get('rating'),
        user_id=current_user.id
    )

    try:
        food_review.create()
        return jsonify(food_review.read())
    except Exception as e:
        return {'message': f'Error saving review: {e}'}, 500

- **Get**: The GET function allows users to retrieve either a specific review (by providing an ID) or all reviews. To improve usability, I joined the User and FoodReview12345 tables, so each review includes the name of the user who posted it. This way, the frontend can easily display a list of food reviews along with the reviewer’s name.

In [ ]:
def get(self):
    review_id = request.args.get('id')

    if review_id:
        review = FoodReview12345.query.get(review_id)
        if not review:
            return {'message': 'Review not found'}, 404
        return jsonify(review.read())

    reviews = db.session.query(FoodReview12345, User).join(User, FoodReview12345.user_id == User.id).all()
    review_list = [{"id": r.FoodReview12345.id, "user_id": r.User._name, "food": r.FoodReview12345.food, "review": r.FoodReview12345.review, "rating": r.FoodReview12345.rating} for r in reviews]

    return jsonify(review_list)

- **Put**: For updating reviews, I designed the PUT function to first check whether a review exists before modifying it. If the review is found, it updates the fields with new data. This prevents users from accidentally updating a nonexistent review and ensures data consistency.

In [ ]:
def put(self):
    data = request.get_json()

    if not data or 'id' not in data:
        return {'message': 'ID is required for updating a review'}, 400

    review = FoodReview12345.query.get(data['id'])
    if not review:
        return {'message': 'Review not found'}, 404

    try:
        review.update(data)
        return jsonify(review.read())
    except Exception as e:
        return {'message': f'Error updating review: {e}'}, 500

- **Delete**: For deleting a review, I ensured that only the correct review is removed by requiring an ID. If the ID matches a review in the database, it is deleted; otherwise, an error message is returned. This protects against accidental deletions and maintains database integrity.

In [ ]:
def delete(self):
    data = request.get_json()

    if not data or 'id' not in data:
        return {'message': 'ID is required for deleting a review'}, 400

    review = FoodReview12345.query.get(data['id'])
    if not review:
        return {'message': 'Review not found'}, 404

    try:
        review.delete()
        return {'message': 'Review deleted successfully'}, 200
    except Exception as e:
        return {'message': f'Error deleting review: {e}'}, 500


---

## Frontend to Backend

- **Review Form**: This form allows users to enter a food name, review text, and rating (1-5) before submitting it to the backend.


In [ ]:
<div class="form-container">
    <h2>Add a Food Review</h2>
    <input type="text" id="foodInput" placeholder="Enter food name" />
    <textarea id="reviewInput" placeholder="Enter your review"></textarea>
    <input type="number" id="ratingInput" placeholder="Enter rating (1-5)" min="1" max="5" />
    <button onclick="addReview()">Submit Review</button>
</div>

- **Fetching and Displaying Reviews**: This function retrieves reviews from the backend and dynamically adds them to the page. Each review is displayed in a card format that includes edit and delete buttons.


In [ ]:
async function fetchReviews() {
    try {
        const response = await fetch(`${pythonURI}/api/food_review_12345_api`);
        const data = await response.json();
        document.getElementById('reviewCount').innerHTML = `<h2>You have ${data.length} food reviews!</h2>`;
        const body = document.getElementById('main-content');
        data.forEach(item => {
            createCard(body, item);
        });
    } catch (error) {
        console.error("Error fetching data:", error);
    }
}


- **Retrieving Reviews (GET Request)**: When a user submits a review, it is validated, saved to the database, and returned as JSON. The API fetches all stored reviews and joins them with user data for display. This ensures each review is linked to the correct user and food item.


In [ ]:
def get(self):
    reviews = db.session.query(FoodReview12345, User).join(User, FoodReview12345.user_id == User.id).all()
    review_list = [{"id": r.FoodReview12345.id, "user_id": r.User._name, "food": r.FoodReview12345.food, "review": r.FoodReview12345.review, "rating": r.FoodReview12345.rating} for r in reviews]
    return jsonify(review_list)

- **Review Model (SQLAlchemy)**: This model defines how reviews are structured and stored in the database. This ensures each review is linked to a user and can be retrieved, updated, or deleted as needed.


In [ ]:
class FoodReview12345(db.Model):
    __tablename__ = 'food_reviews_12345'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    food = db.Column(db.String(255), nullable=False)
    review = db.Column(db.String(255), nullable=False)
    rating = db.Column(db.String(255), nullable=False)

    def create(self):
        try:
            db.session.add(self)
            db.session.commit()
        except Exception as e:
            db.session.rollback()
            logging.warning(f"Error saving review for '{self.food}': {str(e)}")
            raise e

This food review system connects the frontend, backend, and database, allowing users to submit, edit, and delete reviews easily. The API handles requests efficiently, and the database ensures data is stored securely. With this setup, users can interact with reviews in real-time while maintaining full control.

---

## Frontend UX Engineer (Role)

- **Color Scheme**: Use a dark background with light blue accents (#add8e6), light blue color used for boxes and tables, (add8e6)

- **Border and Rounded Elements**: 



In [ ]:
border-radius: 25px; /* Rating bar */
border-radius: 5px; /* Buttons, input fields */

- **Button Styling**: 

In [ ]:
.action-btn, .create-rating-btn {
    background-color: #FFD700;
    color: black;
    border-radius: 5px;
}
.action-btn:hover, .create-rating-btn:hover {
    background-color: #C5B358;
}

- **Text Styling**: 

In [ ]:
color: white; /* General text */
color: #add8e6; /* Table headers */

- **Boxt and Table Styling**: 

In [ ]:
.ratings-table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
    background-color: black;
    color: white;
}

In all, as a frontend UX engineer, I matched up the styling of all pages to look similar and more cohesive. I also tried my best to make the website look as good as possible.

---